In [6]:
import mysql.connector
from mysql.connector import errorcode
import pandas as pd
import numpy as np

In [7]:
# Leemos el csv para dividirlo en 2 para diseño de la base de 

datos = pd.read_csv('Datos/datos_empresa_nonulos.csv', index_col=0)

datos['id_encuesta'] = range(1,len(datos)+1)

In [8]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1614 entries, 0 to 1613
Data columns (total 31 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   age                       1614 non-null   int64  
 1   attrition                 1614 non-null   object 
 2   businesstravel            1614 non-null   object 
 3   dailyrate                 1614 non-null   float64
 4   department                1614 non-null   object 
 5   distancefromhome          1614 non-null   int64  
 6   education                 1614 non-null   int64  
 7   environmentsatisfaction   1614 non-null   int64  
 8   gender                    1614 non-null   object 
 9   hourlyrate                1614 non-null   float64
 10  jobinvolvement            1614 non-null   int64  
 11  joblevel                  1614 non-null   int64  
 12  jobrole                   1614 non-null   object 
 13  jobsatisfaction           1614 non-null   int64  
 14  maritalstatus

In [9]:
datos.head()

,age,attrition,businesstravel,dailyrate,department,distancefromhome,education,environmentsatisfaction,gender,hourlyrate,...,stockoptionlevel,totalworkingyears,trainingtimeslastyear,worklifebalance,yearsatcompany,yearssincelastpromotion,yearswithcurrmanager,datebirth,remotework,id_encuesta
0,51,No,Travel rarely,684.0,Unknown,6,3,1,M,51.0,...,0,10.0,5,3.0,20,15,15,1972,Yes,1
1,52,No,Travel rarely,699.0,Unknown,1,4,3,M,65.0,...,1,34.0,5,3.0,33,11,9,1971,Yes,2
2,42,No,Travel rarely,532.0,Research & development,4,2,3,M,58.0,...,0,22.0,3,3.0,22,11,15,1981,Yes,3
3,47,No,Travel rarely,359.0,Unknown,2,4,1,F,82.0,...,2,10.0,2,3.0,20,5,6,1976,No,4
4,46,No,Travel rarely,1319.0,Unknown,3,3,1,F,45.0,...,1,10.0,5,3.0,19,2,8,1977,No,5


In [10]:
#Creamos el primer data frame filtrando las columnas de la tabla empleados

empleados = datos[['age', 'attrition', 'businesstravel', 'dailyrate', 'department',
       'distancefromhome', 'education', 'gender',
       'hourlyrate',  'joblevel', 'jobrole',
       'maritalstatus','monthlyrate',
       'numcompaniesworked', 'overtime', 'percentsalaryhike',
       'performancerating',  'stockoptionlevel',
       'totalworkingyears', 'trainingtimeslastyear', 'worklifebalance',
       'yearsatcompany', 'yearssincelastpromotion', 'yearswithcurrmanager',
       'datebirth', 'remotework','id_encuesta']]


#empleados.set_index('id_encuesta', inplace=True)

empleados.to_csv('Datos/empleados.csv')

empleados.head(5)

empleados

,age,attrition,businesstravel,dailyrate,department,distancefromhome,education,gender,hourlyrate,joblevel,...,stockoptionlevel,totalworkingyears,trainingtimeslastyear,worklifebalance,yearsatcompany,yearssincelastpromotion,yearswithcurrmanager,datebirth,remotework,id_encuesta
0,51,No,Travel rarely,684.0,Unknown,6,3,M,51.0,5,...,0,10.0,5,3.0,20,15,15,1972,Yes,1
1,52,No,Travel rarely,699.0,Unknown,1,4,M,65.0,5,...,1,34.0,5,3.0,33,11,9,1971,Yes,2
2,42,No,Travel rarely,532.0,Research & development,4,2,M,58.0,5,...,0,22.0,3,3.0,22,11,15,1981,Yes,3
3,47,No,Travel rarely,359.0,Unknown,2,4,F,82.0,4,...,2,10.0,2,3.0,20,5,6,1976,No,4
4,46,No,Travel rarely,1319.0,Unknown,3,3,F,45.0,4,...,1,10.0,5,3.0,19,2,8,1977,No,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1609,36,Yes,Travel rarely,530.0,Unknown,3,1,M,51.0,3,...,1,10.0,6,3.0,16,3,7,1987,No,1610
1610,45,No,Non travel,805.0,Unknown,4,2,M,57.0,2,...,0,10.0,5,2.0,9,0,8,1978,Yes,1611
1611,39,No,Travel rarely,903.0,Unknown,13,5,M,41.0,3,...,0,9.0,3,3.0,8,0,7,1984,Yes,1612
1612,36,No,Non travel,1229.0,Unknown,8,4,M,84.0,2,...,2,12.0,3,3.0,7,0,7,1987,Yes,1613


In [11]:
# Creamos el dataframe de satisfacion



satisfaccion = datos[['environmentsatisfaction', 'jobinvolvement', 'jobsatisfaction', 'relationshipsatisfaction', 'id_encuesta']]


# satisfaccion.set_index('id_encuesta', inplace=True)

satisfaccion.to_csv('Datos/satisfaccion.csv')

satisfaccion

,environmentsatisfaction,jobinvolvement,jobsatisfaction,relationshipsatisfaction,id_encuesta
0,1,3,3,3,1
1,3,2,3,1,2
2,3,3,4,4,3
3,1,3,3,2,4
4,1,4,1,4,5
...,...,...,...,...,...
1609,3,2,4,1,1610
1610,3,3,2,2,1611
1611,13,4,3,4,1612
1612,1,3,4,4,1613


In [12]:
# This function creates the BDD. It connects to the server and if it works it creates it, if it fails it tells you the failure.
def create_bdd():
    cnx = mysql.connector.connect(user='root', password='AlumnaAdalab', host='127.0.0.1')
    mycursor = cnx.cursor()

    try:
        mycursor.execute("CREATE DATABASE abc_brain_drain") # Here, the name of the DB is indicated after the CREATE DATABASE operator.
        print("DATABASE CREATED UNDER THE NAME: abc_brain_drain 🆗")
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)
    finally:
        cnx.commit()
        mycursor.close()
        cnx.close()

In [13]:
# This function creates the tables. It connects to the server and if it works it creates it, if it fails it tells you the failure.
def create_tables():
    cnx = mysql.connector.connect(user='root', password='AlumnaAdalab', host='127.0.0.1', database='abc_brain_drain')
    mycursor = cnx.cursor()

    # Query list where all tables are created
    queries = [
        """CREATE TABLE empleados (
    id_encuesta INT PRIMARY KEY,
    age INT,
    attrition VARCHAR(100),
    dailyrate FLOAT,
    businesstravel VARCHAR(100),
    department VARCHAR(100),
    distancefromhome INT,
    education INT,
    gender VARCHAR(10),
    hourlyrate FLOAT,
    joblevel INT,
    jobrole VARCHAR(100),
    maritalstatus VARCHAR(50),
    monthlyrate INT,
    numcompaniesworked INT,
    overtime VARCHAR(50),
    percentsalaryhike INT,
    performancerating FLOAT,
    stockoptionlevel INT,
    totalworkingyears FLOAT,
    trainingtimeslastyear INT,
    worklifebalance FLOAT,
    yearsatcompany INT,
    yearssincelastpromotion INT,
    yearswithcurrmanager INT,
    datebirth YEAR,
    remotework VARCHAR(10)
);""",
        """CREATE TABLE satisfaccion (
    id_encuesta INT PRIMARY KEY,
    environmentsatisfaction INT,
    jobinvolvement INT,
    jobsatisfaction INT,
    relationshipsatisfaction INT,
    FOREIGN KEY (id_encuesta) REFERENCES empleados(id_encuesta)
);"""
    ]

    # I have created this list of table names so that I can iterate over it and it will print their names appropriately as it makes them.
    table_names = [
        'empleados',
        'satisfaccion'
    ]

    # The zip is a function that groups lists that have multiple elements into tuples. 
    # This then allows you to iterate through the names in the print.
    for query, table_name in zip(queries, table_names):
        try:
            mycursor.execute(query) 
            print(f"TABLE CREATED UNDER THE NAME: {table_name} 🆗")
        
        except mysql.connector.Error as err:
            print(err)
            print("Error Code:", err.errno)
            print("SQLSTATE", err.sqlstate)
            print("Message", err.msg)

    cnx.commit()
    mycursor.close()
    cnx.close()

In [14]:
create_bdd()

DATABASE CREATED UNDER THE NAME: abc_brain_drain 🆗


In [15]:
create_tables()

TABLE CREATED UNDER THE NAME: empleados 🆗
TABLE CREATED UNDER THE NAME: satisfaccion 🆗


In [20]:
def insert_empleados():
    # Connection
    cnx = mysql.connector.connect(user='root', password='AlumnaAdalab', host='127.0.0.1', database='abc_brain_drain')

    # Cursor creation
    mycursor = cnx.cursor()

    try: 
        # Read CSV file
        df = pd.read_csv('Datos/empleados.csv') 

        # SQL to insert data. You must indicate the name of the columns in the first parenthesis and in values enter the number of columns with %s
        sql_insert_query = """ INSERT INTO empleados (age, attrition, businesstravel, dailyrate, department,
       distancefromhome, education, gender, hourlyrate, joblevel,
       jobrole, maritalstatus, monthlyrate,
       numcompaniesworked, overtime, percentsalaryhike,
       performancerating, stockoptionlevel, totalworkingyears,
       trainingtimeslastyear, worklifebalance, yearsatcompany,
       yearssincelastpromotion, yearswithcurrmanager, datebirth,
       remotework, id_encuesta) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""

        # Iterate over each row of the DataFrame and insert the data for each column.
        for _, row in df.iterrows():
            mycursor.execute(sql_insert_query, (row['age'], row['attrition'], row['businesstravel'], row['dailyrate'], row['department'],
                    row['distancefromhome'], row['education'], row['gender'], row['hourlyrate'], row['joblevel'],
                    row['jobrole'], row['maritalstatus'], row['monthlyrate'],
                    row['numcompaniesworked'], row['overtime'], row['percentsalaryhike'],
                    row['performancerating'], row['stockoptionlevel'], row['totalworkingyears'],
                    row['trainingtimeslastyear'], row['worklifebalance'], row['yearsatcompany'],
                    row['yearssincelastpromotion'], row['yearswithcurrmanager'], row['datebirth'],
                    row['remotework'], row['id_encuesta'] )) 

        # Confirm changes with commit
        cnx.commit() 
        print(f"{mycursor.rowcount} registro(s) insertado(s)📝") 

    except mysql.connector.Error as err: 
        print(f"Error al insertar en la base de datos: {err} ❌") 

    finally: 
        if cnx.is_connected(): 
            mycursor.close() 
            cnx.close() 
            print("Conexión a la base de datos cerrada🔚")

In [21]:
insert_empleados()

1 registro(s) insertado(s)📝
Conexión a la base de datos cerrada🔚


In [24]:
def insert_satisfaction():
    # Connection
    cnx = mysql.connector.connect(user='root', password='AlumnaAdalab', host='127.0.0.1', database='abc_brain_drain')

    # Cursor creation
    mycursor = cnx.cursor()

    try: 
        # Read CSV file
        df = pd.read_csv('Datos/satisfaccion.csv') 
        
        # SQL to insert data. You must indicate the name of the columns in the first parenthesis and in values enter the number of columns with %s
        sql_insert_query = """ INSERT INTO satisfaccion (environmentsatisfaction, jobinvolvement, jobsatisfaction, relationshipsatisfaction, id_encuesta) 
        VALUES (%s, %s, %s, %s, %s)"""

        # Iterate over each row of the DataFrame and insert the data for each column.
        for _, row in df.iterrows():
            mycursor.execute(sql_insert_query, (row['environmentsatisfaction'], row['jobinvolvement'], row['jobsatisfaction'], row['relationshipsatisfaction'], row['id_encuesta'] )) 

        # Confirm changes with commit
        cnx.commit() 
        print(f"{mycursor.rowcount} registro(s) insertado(s)📝") 

    except mysql.connector.Error as err: 
        print(f"Error al insertar en la base de datos: {err} ❌") 

    finally: 
        if cnx.is_connected(): 
            mycursor.close() 
            cnx.close() 
            print("Conexión a la base de datos cerrada🔚")

In [25]:
insert_satisfaction()

Conexión a la base de datos cerrada🔚


TypeError: CMySQLCursor.execute() takes from 2 to 4 positional arguments but 7 were given

In [27]:
def insertsatisfaction():
    # Connection
    cnx = mysql.connector.connect(user='root', password='AlumnaAdalab', host='127.0.0.1', database='abc_brain_drain')

    # Cursor creation
    mycursor = cnx.cursor()

    try: 
        # Read CSV file
        df = pd.read_csv('Datos/satisfaccion.csv')

        # SQL query to insert data
        sql_insert_query = """ 
        INSERT INTO satisfaccion (environmentsatisfaction, jobinvolvement, jobsatisfaction, relationshipsatisfaction, id_encuesta) 
        VALUES (%s, %s, %s, %s, %s)"""

        # Iterate over each row of the DataFrame and insert the data for each column.
        for index, row in df.iterrows():
            # Convert numpy.int64 to Python int if necessary
            data = (
                int(row['environmentsatisfaction']),
                int(row['jobinvolvement']),
                int(row['jobsatisfaction']),
                int(row['relationshipsatisfaction']),
                int(row['id_encuesta'])
            )
            mycursor.execute(sql_insert_query, data)

        # Confirm changes with commit
        cnx.commit()
        print(f"{mycursor.rowcount} registro(s) insertado(s)📝") 

    except mysql.connector.Error as err: 
        print(f"Error al insertar en la base de datos: {err} ❌") 

    finally: 
        if cnx.is_connected(): 
            mycursor.close() 
            cnx.close() 
            print("Conexión a la base de datos cerrada🔚")


insertsatisfaction()

1 registro(s) insertado(s)📝
Conexión a la base de datos cerrada🔚
